In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules/").absolute().__str__())

import ir_measures
from ir_measures import *
from data_utils import load_dataset
import json
import pandas as pd
import collections

In [2]:
data_test = json.loads(Path("../tests/scriptstests/data/evaluation_test/evaluation_test_gold.json").read_text())
df_gold = pd.read_json(data_test['df_gold_eval'])
df_gold['gold_eval_df'] = df_gold.apply(lambda x: pd.DataFrame(data=x['gold_eval_df']), axis=1)
df_gold.head()

,query,description,context,prompt,num_tokens,start_with_SELECT,query_templated,result,full_answer,is_skipped,is_prompt_too_long,execution,executed_query,input,target_raw,target_template,gold_eval,gold_get_nested_values,gold_eval_df,gold_id_columns
2300,SELECT ?l ?lemma WHERE {\n?l dct:language wd:Q...,Danish idioms,,"<s>[INST] QUERY=""SELECT ?l ?lemma WHERE {\n?l ...",157,True,PREFIX dct: <http://purl.org/dc/terms/>\nPREFI...,"[""Find Danish idioms and their corresponding l...","{'content': ' 1. ""Find Danish idioms and their...",False,False,"[{'l': {'type': 'uri', 'value': 'http://www.wi...",PREFIX dct: <http://purl.org/dc/terms/>\nPREFI...,"[""Find Danish idioms and their corresponding l...",SELECT ?l ?lemma WHERE {\n?l dct:language wd:Q...,PREFIX dct: <http://purl.org/dc/terms/>\nPREFI...,"[{'l': {'type': 'uri', 'value': 'http://www.wi...","[http://www.wikidata.org/entity/L347127, have ...",l ...,{'l': {'0': 'http://www.wikidata.org/entity/L3...
2628,SELECT ?item ?itemLabel ?constituency ?constit...,List of candidates for the next French legisla...,,"<s>[INST] QUERY=""SELECT ?item ?itemLabel ?cons...",279,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Write a SparQL query to retrieve the name, c...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Write a SparQL query to retrieve the name, c...",SELECT ?item ?itemLabel ?constituency ?constit...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...","[http://www.wikidata.org/entity/Q1457778, http...",item ...,{'item': {'0': 'http://www.wikidata.org/entity...
1806,SELECT DISTINCT ?item ?itemLabel ?image WHERE ...,100 random paintings related to Haarlem,,"<s>[INST] QUERY=""SELECT DISTINCT ?item ?itemLa...",247,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Find and list the titles, labels, and a rand...","{'content': ' 1. ""Find and list the titles, la...",False,False,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Find and list the titles, labels, and a rand...",SELECT DISTINCT ?item ?itemLabel ?image WHERE ...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...","[http://www.wikidata.org/entity/Q57517799, Hou...",item ...,{'item': {'0': 'http://www.wikidata.org/entity...
1345,SELECT ?item ?itemLabel ?countryLabel ?coords ...,Natural arches around the globe] ([ source,,"<s>[INST] QUERY=""SELECT ?item ?itemLabel ?coun...",264,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Find natural arches from around the world, i...","{'content': ' 1. ""Find natural arches from aro...",False,False,"[{'item': {'type': 'uri', 'value': 'http://www...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Find natural arches from around the world, i...",SELECT ?item ?itemLabel ?countryLabel ?coords ...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'item': {'type': 'uri', 'value': 'http://www...","[http://www.wikidata.org/entity/Q124101953, Po...",item ...,{'item': {'0': 'http://www.wikidata.org/entity...
251,SELECT ?person ?personLabel ?personDescription...,,Ashmolean Museum\nAshmolean people associated ...,"<s>[INST] QUERY=""SELECT ?person ?personLabel ?...",407,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Retrieve the names, labels, descriptions, an...","{'content': ' 1. ""Retrieve the names, labels, ...",False,False,"[{'person': {'type': 'uri', 'value': 'http://w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[""Retrieve the names, labels, descriptions, an...",SELECT ?person ?personLabel ?personDescription...,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[{'person': {'type': 'uri', 'value': 'http://w...","[http://www.wikidata.org/entity/Q1375891, Tani...",person ...,{'person': {'0': 'http://www.wikidata.org/enti...


In [3]:
df_gold_eval = df_gold['gold_eval_df']
df_gold_eval.iloc[0]

,l,lemma
0,http://www.wikidata.org/entity/L347127,have alt at vinde og intet at tabe
1,http://www.wikidata.org/entity/L347132,stå ned i stænger
2,http://www.wikidata.org/entity/L347373,på grund af
3,http://www.wikidata.org/entity/L347542,stå ned
4,http://www.wikidata.org/entity/L589764,få nok
5,http://www.wikidata.org/entity/L589769,tak for kaffe
6,http://www.wikidata.org/entity/L589772,ude og sejle
7,http://www.wikidata.org/entity/L592109,sove i timen
8,http://www.wikidata.org/entity/L314527,"alt godt kommer til den, der venter"
9,http://www.wikidata.org/entity/L347130,af den gamle skole


In [4]:
df_gold_eval.iloc[0].columns

Index(['l', 'lemma'], dtype='object')

In [5]:
from evaluation_utils import cross_product_func

def make_qrel_namedtuple_from_element(qid:str, element, relevance:int):
    if not isinstance(relevance, int):
        raise TypeError()
    if relevance != 0 and relevance != 1:
        raise ValueError()
    
    value = None
    if isinstance(element, str):
        value = element
    elif isinstance(element, int) or isinstance(element, float):
        value = str(element)
    else:
        raise NotImplementedError()
    return ir_measures.Qrel(query_id=qid, doc_id=value, relevance=relevance)

def transform_df_column_into_qrel_list(qid:str, column: pd.Series):
    return column.map(lambda x: make_qrel_namedtuple_from_element(qid, x, 1)).to_list()

def transform_list_into_qrel_list(qid:str, column: list):
    return [make_qrel_namedtuple_from_element(qid, x, 1) for x in column]

def transform_df_into_qrel_list(df: pd.DataFrame, output:str="aggregated"):
    if not isinstance(output, str):
        raise TypeError()
    if output not in ['aggregated', 'list_of_list']:
        raise ValueError()
    
    if not isinstance(df, pd.DataFrame):
        raise TypeError()
    
    if df.empty:
        return []
    
    columns = list(df.columns)
    results = []
    for col in columns:
        if output == "aggregated":
            results += transform_df_column_into_qrel_list(qid="0", column=df[col])
        elif output == "list_of_list":
            results.append(transform_df_column_into_qrel_list(qid="0", column=df[col]))
            
    return results

def make_scoreddoc_namedtuple_from_element(qid:str, element, score:float):
    if not isinstance(score, int) and not isinstance(score, float):
        raise TypeError()
    
    value = None
    if isinstance(element, str):
        value = element
    elif isinstance(element, int) or isinstance(element, float):
        value = str(element)
    else:
        raise NotImplementedError()
    return ir_measures.ScoredDoc(query_id=qid, doc_id=value, score=score)

def transform_df_column_into_run_list(qid:str, column: pd.Series):
    return column.map(lambda x: make_scoreddoc_namedtuple_from_element(qid, x, 1)).to_list()

def transform_list_into_run_list(qid:str, column: list):
    return [make_scoreddoc_namedtuple_from_element(qid, x, 1) for x in column]

def transform_df_into_run_list(df: pd.DataFrame, output:str="aggregated"):
    if not isinstance(output, str):
        raise TypeError()
    if output not in ['aggregated', 'list_of_list']:
        raise ValueError()
    
    if not isinstance(df, pd.DataFrame):
        raise TypeError()
    
    if df.empty:
        return []
    
    columns = list(df.columns)
    results = []
    for col in columns:
        if output == "aggregated":
            results += transform_df_column_into_run_list(qid="0", column=df[col])
        elif output == "list_of_list":
            results.append(transform_df_column_into_run_list(qid="0", column=df[col]))
            
    return results

def cross_ir_measures_wrapper(y_true, y_pred, f):
    return f(transform_list_into_qrel_list(qid="0", column=y_true), transform_list_into_run_list(qid="0", column=y_pred))

ComputeMetricsResult = collections.namedtuple('ComputeMetricsResult', ['mean_average_precision', 'precision_k', 'recall_k', 'mean_reciprocal_rank', 'k'])
def compute_metrics_for_two_df(results, gold, k:int=5, how="cross"):
    if not isinstance(how, str):
        raise TypeError()
    
    if how not in ['nested', 'cross']:
        raise ValueError()
    
    if not isinstance(results, pd.DataFrame):
        raise TypeError()
    if not isinstance(gold, pd.DataFrame):
        raise TypeError()
    
    if not isinstance(k, int):
        raise TypeError()
    
    results_dict = dict()
    results_dict['k'] = k
    if how == "cross":
        results_dict['mean_average_precision'] = cross_product_func(func=cross_ir_measures_wrapper, y_true=gold, y_pred=gold, f=ir_measures.AP.calc_aggregate)
        results_dict['precision_k'] = cross_product_func(func=cross_ir_measures_wrapper, y_true=gold, y_pred=gold, f=ir_measures.parse_measure(f'P@{k}').calc_aggregate)
        results_dict['recall_k'] = cross_product_func(func=cross_ir_measures_wrapper, y_true=gold, y_pred=gold, f=ir_measures.parse_measure(f'Success@{k}').calc_aggregate)
        results_dict['mean_reciprocal_rank'] = cross_product_func(func=cross_ir_measures_wrapper, y_true=gold, y_pred=gold, f=ir_measures.parse_measure(f'RR@{k}').calc_aggregate)
        
    return ComputeMetricsResult(
        mean_average_precision=results_dict['mean_average_precision'],
        precision_k=results_dict['precision_k'],
        recall_k=results_dict['recall_k'],
        mean_reciprocal_rank=results_dict['mean_reciprocal_rank'],
        k=results_dict['k']
    )

In [6]:
qrels = transform_df_into_qrel_list(df_gold_eval.iloc[0])

In [7]:
runs = transform_df_into_run_list(df_gold_eval.iloc[0])

In [8]:
AP = ir_measures.AP

AP.calc_aggregate(qrels=qrels, run=runs)

1.0

In [9]:
P = ir_measures.P

(P@5).calc_aggregate(qrels=qrels, run=runs)

1.0

In [10]:
R = ir_measures.Success

(R@5).calc_aggregate(qrels=qrels, run=runs)

1.0

In [11]:
RR = ir_measures.RR

(RR@5).calc_aggregate(qrels=qrels, run=runs)

1.0

In [12]:
results = compute_metrics_for_two_df(results=df_gold_eval.iloc[0], gold=df_gold_eval.iloc[0])
print(f"{results.mean_average_precision=}")
print(f"{results.precision_k=}")
print(f"{results.recall_k=}")
print(f"{results.mean_reciprocal_rank=}")
print(f"{results.k=}")

results.mean_average_precision=1.0
results.precision_k=1.0
results.recall_k=1.0
results.mean_reciprocal_rank=1.0
results.k=5
